In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import xlwt
import pandas as pd
import numpy as np

month2num = dict()
num2month = dict()
months = ['January', 'February','March','April','May','June','July','August','September','October','November','December']
months_abb = ['Jan', 'Feb', 'Mar']
nums = range(1,13)
for i in range(12):
    month2num[months[i]] = nums[i]
    num2month[nums[i]] = months[i]

#### Simulate selecting date on the date picker of the webpage

In [2]:
def choose(calendar_dates, input_dates):
    pattern_attr = ''
    cal_day, cal_month, cal_year = calendar_dates.split('/') 
    input_day, input_month, input_year = input_dates.split('/') 
    
    click_times = 12*(int(input_year) - int(cal_year)) + int(input_month) - int(cal_month)
    print(click_times)
    pattern_date = num2month[int(input_month)][:3] + ' ' + input_day + ' ' + input_year
    print(pattern_date)
    if click_times > 0:
        for i in range(click_times):
            time.sleep(2)
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div._1HphCM4i > div:nth-child(2) > div > div:nth-child(2) > div > div.hotels-calendar-calendar__checkInContainer--2A6dC > div > div.hotels-calendar-calendar__navBar--3sFon > button.ui_icon.single-chevron-right.hotels-calendar-calendar__navButton--21dqp.hotels-calendar-calendar__next--1svjm').click()
            print('click')
    else:
        for i in range(abs(click_times)):
            time.sleep(2)
            driver.find_element_by_css_selector('#BODY_BLOCK_JQUERY_REFLOW > div._1HphCM4i > div:nth-child(2) > div > div:nth-child(2) > div > div.hotels-calendar-calendar__checkInContainer--2A6dC > div > div.hotels-calendar-calendar__navBar--3sFon > button.ui_icon.single-chevron-left.hotels-calendar-calendar__navButton--21dqp.hotels-calendar-calendar__prev--3y9l4').click()
            print('click')
    soup = BeautifulSoup(driver.page_source, 'lxml')
    col_blocks = soup.find_all('div',{'class' : 'hotels-calendar-calendar__week--3Swx8'})
    for b in col_blocks:
        individual_blocks = b.find_all('div', {'aria-disabled' : 'false'}) 
        for i in individual_blocks:
            if pattern_date in str(i):
                pattern_attr = i.get('aria-label')
    return pattern_attr

def calendar(wait, driver, checkin_date, checkout_date):
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "hotels-calendar-calendar__caption--1vILf")))
    cal_dates = driver.find_element_by_class_name("hotels-calendar-calendar__caption--1vILf").text
    print(cal_dates)
    cal_month, cal_year = cal_dates.split()
    cal_month = month2num[cal_month]
    cal_month = str(cal_month)
    if(len(cal_month) == 1):
        cal_month = '0' + str(cal_month)
    cal_dates = '10/'+cal_month+'/'+cal_year 
    print(cal_dates)

    pattern_in = choose(cal_dates, checkin_date)
    pattern_in = "div[aria-label='" + pattern_in +"']"  
    print(pattern_in)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, pattern_in))).click()
    print('click checkin date')

    pattern_out = choose(checkin_date, checkout_date)
    pattern_out = "div[aria-label='" + pattern_out +"']"  
    # driver.find_element_by_css_selector(pattern_out).click()
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, pattern_out))).click()
    print('click checkout date')

### url scraper.py

##### Crawl the information of each hotel in the list (including url) 

In [32]:
# step1: get into the target url
domain = 'https://www.tripadvisor.com'
target_url = 'https://www.tripadvisor.com/Hotels/'

opt = webdriver.ChromeOptions()
# opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) # Turn on developer mode
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options=opt)

wait = WebDriverWait(driver, 15)
driver.get(target_url)

# step2: input destination
input_str = 'London'
print('You want to go to {}!'.format(input_str))
c = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_hotels_0"]/div[2]/div[1]/div')))
c.click()
input_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'typeahead_input')))
input_box.clear()
input_box.send_keys(input_str)
input_box.send_keys(Keys.ENTER)

# confirm the number of pages
total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#taplc_main_pagination_bar_dusty_hotels_resp_0 > div > div > div > div > a.pageNum.last.cx_brand_refresh_phase2')))
page_num  = min(int(total.text), 10)
print("Total number of page: {}".format(page_num))

# step3: select dates to view deals
checkin_date = '12/12/2020'
checkout_date = '16/12/2020'
calendar(wait, driver, checkin_date, checkout_date)

# step 4: Crawl the information of each hotel in the list (including url)
with open('hotel_url_london.csv', 'a') as csvfile:
    fieldnames = ['hotel_id', 'hotel_name', 'n_comment', 'n_star', 'url', 'Price']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames, )
    writer.writeheader()
    index = 0
    for p in range(page_num):
        
        print('the number of page = {0}/{1}'.format(p + 1, page_num))
        soup = BeautifulSoup(driver.page_source, 'lxml')
        hotel_blocks = soup.find_all('div', {"class" : "prw_rup prw_meta_hsx_responsive_listing ui_section listItem"})
    
        for element in hotel_blocks:
            index += 1
            hotel_name = element.find('div', {"class" : "listing_title"}).text
            url = domain+element.find('div', {"class" : "listing_title"}).find('a').get('href')
            
            if str(type(element.find('a', {"data-clicksource" : "ReviewCount"}))) != "<class 'NoneType'>" and str(type(element.find('a', {"data-clicksource" : "BubbleRating"}))) != "<class 'NoneType'>":
                price = element.find('div', {"class" : "price __resizeWatch"}).text
                n_comment = element.find('a', {"data-clicksource" : "ReviewCount"}).text
                n_comment = re.sub('[^0-9,]', "", n_comment).replace(',','')
                n_star = element.find('a', {"data-clicksource" : "BubbleRating"}).attrs['alt']
                writer.writerow(
                    {
                                'hotel_id' : index,
                                'hotel_name' : hotel_name,
                                'n_comment' : n_comment,
                                'n_star' : n_star,
                                'url' : url,
                                'Price' : price
                    }
                )
            else: break
                    
            
        time.sleep(3)
#         wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page))).click()
        if page_num - 1 == p:
            break
            
        next_page = domain + soup.find(class_ = "nav next ui_button primary cx_brand_refresh_phase2").get('href')
        driver.get(next_page)
        print('click')
        time.sleep(5)
driver.quit()

You want to go to London!
Total number of page: 10
April 2020
10/04/2020
8
Dec 12 2020
click
click
click
click
click
click
click
click
div[aria-label='Sat Dec 12 2020']
click checkin date
0
Dec 16 2020
click checkout date
the number of page = 1/10
click
the number of page = 2/10
click
the number of page = 3/10
click
the number of page = 4/10
click
the number of page = 5/10
click
the number of page = 6/10
click
the number of page = 7/10
click
the number of page = 8/10
click
the number of page = 9/10
click
the number of page = 10/10


##### Extract url from crawled hotel information

In [99]:
# reading datafile
df = pd.read_csv('hotel_url_london.csv')
df = df[df.n_comment > 10]
total_hotels = len(df)

# initialization
fieldnames = [
                'hotel_id', 'address', 'rank', 'Description','Amenities',
                'score', 'Location', 'Cleanliness','Service', 'Value',
                'n_Excellent', 'n_VeryGood', 'n_Average', 'n_Poor', 'n_Terrible'
             ]
df_content = pd.DataFrame(columns = fieldnames)

def rating_processing(tmp):
    tmp = int(re.sub('[^0-9,]', "", tmp).replace(',',''))
    return tmp

def score_processing(tmp):
    tmp = float(re.sub('[^0-9,]', "", tmp)) / 10 
    return tmp

debug = False
if debug:
    limit = 3
else:
    limit = total_hotels + 1

# browser setting
opt = webdriver.ChromeOptions()
opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) # Turn on developer mode
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options=opt)
    
for idx, u in enumerate(df['url'][:limit]):
    hotel_id = df['hotel_id'][idx]
    print('process = {}/{}'.format(idx + 1, total_hotels))
    driver.get(u)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # --------------------------------
    # infomation
    try:
        address = soup.find_all('span',
                                {'class' : "public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI"})[0].text
    except:
        address = 'N/A'
    
    try:
        rank = int(re.sub('[^0-9,]', "", soup.find_all('b',{'class', 'rank'})[0].text))
    except:
        rank = 'N/A'
        
    try: 
        desc = soup.find_all('div',{'class' : "cPQsENeY"})[0].text
    except:
        desc = 'N/A'
    
    # --------------------------------
    # property amenity
    try:
        amenity_blocks = soup.find_all('div',{'class' : 'hotels-hr-about-amenities-Amenity__amenity--3fbBj'})
        amenity = []
        for amen in amenity_blocks:
            amenity.append(amen.text)
    except:
        amenity = []
    
    # --------------------------------
    # evaluation
    try:
        avg_score = score_processing(soup.find_all('span',{'class', "hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA"})[0].text)
    except:
        avg_score = 'N/A'
    
    
    try:
        scores_soup = soup.find_all('div', {'class': "hotels-hotel-review-about-with-photos-Reviews__subratingRow--2u0CJ"})
        scores = []
        for s in scores_soup:
            scores.append(score_processing(s.span.attrs['class'][1]))
    except:
        scores = []
        for i in range(4):
            scores.append('N/A')
    
    # --------------------------------
    # reviews score
    try:
        rates = []
        rates_soup = soup.find_all('span',{'class' : 'location-review-review-list-parts-ReviewRatingFilter__row_num--3cSP7'})
        for r in rates_soup:
            rates.append(rating_processing(r.text))
        
        
    except:
        rates = []
        for i in range(5):
            rates.append('N/A')
    
    
    
    df_content.loc[idx] = [hotel_id, address, rank, desc, amenity,
                           avg_score, scores[0], scores[1], scores[2], scores[3], 
                           rates[0], rates[1], rates[2], rates[3], rates[4]]
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use setter for headless property instead of set_headless


process = 1/320
process = 2/320
process = 3/320
process = 4/320
process = 5/320
process = 6/320
process = 7/320
process = 8/320
process = 9/320
process = 10/320
process = 11/320
process = 12/320
process = 13/320
process = 14/320
process = 15/320
process = 16/320
process = 17/320
process = 18/320
process = 19/320
process = 20/320
process = 21/320
process = 22/320
process = 23/320
process = 24/320
process = 25/320
process = 26/320
process = 27/320
process = 28/320
process = 29/320
process = 30/320
process = 31/320
process = 32/320
process = 33/320
process = 34/320
process = 35/320
process = 36/320
process = 37/320
process = 38/320
process = 39/320
process = 40/320
process = 41/320
process = 42/320
process = 43/320
process = 44/320
process = 45/320
process = 46/320
process = 47/320
process = 48/320
process = 49/320
process = 50/320
process = 51/320
process = 52/320
process = 53/320
process = 54/320
process = 55/320
process = 56/320
process = 57/320
process = 58/320
process = 59/320
proces

TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.001
  (Session info: headless chrome=80.0.3987.163)


In [101]:
df_content_copy = df_content

In [103]:
df_content.to_csv('hotel_info.csv.csv')